# Results comparison between Trajectron++ and our method
Comparing the results of our method with the Trajectron++ paper's method. The goal of the comparison is to evaluate Trajectron++ on a different dataset than it was trained on to allow more fair comparison.

In [159]:
import sys
import os
import pandas as pd
import numpy as np
import generative_model
import matplotlib.pyplot as plt
from tqdm import tqdm
import csv
np.random.seed(1)

## Helper functions

In [160]:
def calculate_FDE(pred_x, pred_y, test_x, test_y):

    final_displacement_x = pred_x[-1] - test_x[-1]
    final_displacement_y = pred_y[-1] - test_y[-1]
    FDE = np.sqrt(final_displacement_x**2 + final_displacement_y**2)
    
    return FDE

def calculate_ADE(pred_x, pred_y, test_x, test_y):
    assert len(pred_x) == len(test_x)
    total_displacement_error = 0
    for point_idx in range(len(test_x)):
        displacement_error = np.sqrt((pred_x[point_idx] - test_x[point_idx])**2 + (pred_y[point_idx] - test_y[point_idx])**2)
        total_displacement_error += displacement_error

    return total_displacement_error/len(pred_x)

## The evaluation logic for Trajectron++ loops over the frames and predicts the future trajectories 
## for each node present in the current frame
## Each node has to have at least 7 historical points and 12 future points
def get_total_predictable_slices(data):
    total_predictable_steps = 0
    for i in pd.unique(data.node_id):
        #print(len(test[test.node_id == i]))
        total_predictable_steps += len(data[data.node_id == i]) - 19
    return total_predictable_steps

In [162]:
def process_data(input_data):
    data = input_data.copy()
    data['frame_id'] = pd.to_numeric(data['frame_id'], downcast='integer')
    data['track_id'] = pd.to_numeric(data['track_id'], downcast='integer')

    data['frame_id'] = data['frame_id'] // 10

    data['frame_id'] -= data['frame_id'].min()

    data['node_type'] = 'PEDESTRIAN'
    data['node_id'] = data['track_id'].astype(str)
    data.sort_values('frame_id', inplace=True)

    data['pos_x'] = data['pos_x'] - data['pos_x'].mean()
    data['pos_y'] = data['pos_y'] - data['pos_y'].mean()
    
    # Select only such nodes which have enough data to predict on (8 historical timesteps, 12 future)
    v = data.node_id.value_counts()
    data = data[data.node_id.isin(v.index[v.gt(19)])]
    
    return data

## Method evaluation logic

In [163]:
def evaluate_our_method(data, params, dataset_title='', clustering_method='KMeans', smoothing=False):
    our_fde_best_of = []
    our_ade_best_of = []
    our_fde_single = []
    our_ade_single = []

    # Loop over the dataset frame by frame
    for frame_id in tqdm(pd.unique(data.frame_id), desc='Ours - ' + dataset_title):

        frame_data = data[data.frame_id == frame_id]
        
        # Loop over all agents in the current frame
        for node_id in pd.unique(frame_data.node_id):
            # Check for 8 points of history for current agent
            if len(data[((data.node_id == node_id) & (data.frame_id <= frame_id))]) >= 8:
                # Check for 12 points of future for current agent
                if len(data[((data.node_id == node_id) & (data.frame_id > frame_id))]) >= 12:
                    node_history_data = data[((data.node_id == node_id) & (data.frame_id <= frame_id) & (data.frame_id >= frame_id-7))]
                    node_gt_data = data[((data.node_id == node_id) & (data.frame_id > frame_id) & (data.frame_id <= frame_id+12))]

                    x_data = list(node_history_data.pos_x)
                    y_data = list(node_history_data.pos_y)
                    assert len(x_data) == 8

                    x_gt = list(node_gt_data.pos_x)
                    y_gt = list(node_gt_data.pos_y)
                    assert len(x_gt) == 12

                    all_pred_x, all_pred_y, weights = generative_model.predict(x_data, y_data, params, trajectory_length=12, clustering_method=clustering_method, smoothing=smoothing)
                    assert len(all_pred_x[0]) == 12
                    
                    # This section is for producing a single trajectory (averaging all representative preds)
                    avg_x = np.average(all_pred_x, axis=0, weights=weights)
                    avg_y = np.average(all_pred_y, axis=0, weights=weights)
                    #avg_x = np.mean(all_pred_x, axis=0)
                    #avg_y = np.mean(all_pred_y, axis=0)
                    avg_fde = calculate_FDE(avg_x, avg_y, x_gt, y_gt)
                    avg_ade = calculate_ADE(avg_x, avg_y, x_gt, y_gt)
                    our_fde_single.append(avg_fde)
                    our_ade_single.append(avg_ade)
                        
                        
                    # This section is for finding the best trajectories out of many in terms of FDE and ADE
                    best_fde = None
                    best_ade = None

                    for i in range(len(all_pred_x)):
                        current_pred_x = all_pred_x[i]
                        current_pred_y = all_pred_y[i]

                        fde = calculate_FDE(current_pred_x, current_pred_y, x_gt, y_gt)
                        if best_fde == None or fde < best_fde:
                            best_fde = fde

                        ade = calculate_ADE(current_pred_x, current_pred_y, x_gt, y_gt)
                        if best_ade == None or ade < best_ade:
                            best_ade = ade

                    our_fde_best_of.append(best_fde)
                    our_ade_best_of.append(best_ade)
                    
    return our_fde_best_of, our_ade_best_of, our_fde_single, our_ade_single


In [164]:
def evaluate_cvm_with_scenarios(data, dataset_title='', history_length=8):
    tot = 0
    our_fde_best_of = []
    our_ade_best_of = []

    for frame_id in tqdm(pd.unique(data.frame_id), desc='CVM - ' + dataset_title):

        frame_data = data[data.frame_id == frame_id]
        #print(frame_data)
        for node_id in pd.unique(frame_data.node_id):
            # Check if at least 7 historical points are present
            # PS: It might be so that the prediction starts at the 8th step instead of 7th? Edited the code to do this at the moment
            if len(data[((data.node_id == node_id) & (data.frame_id <= frame_id))]) >= 8:
                # Not sure why there has to be more than 12 frames to the future (at least 13) but it's the
                # only way to get the number of trajectron++ eval predictions to match up
                if len(data[((data.node_id == node_id) & (data.frame_id > frame_id))]) >= 12:
                    tot += 1
                    node_history_data = data[((data.node_id == node_id) & (data.frame_id <= frame_id) & (data.frame_id >= frame_id-7))]
                    node_gt_data = data[((data.node_id == node_id) & (data.frame_id > frame_id) & (data.frame_id <= frame_id+12))]

                    x_data = list(node_history_data.pos_x)
                    y_data = list(node_history_data.pos_y)
                    assert len(x_data) == 8

                    x_gt = list(node_gt_data.pos_x)
                    y_gt = list(node_gt_data.pos_y)
                    assert len(x_gt) == 12

                    all_pred_x, all_pred_y = [], []
                    
                    # CVM
                    for i in range(20):
                        history_x = x_data[-history_length:]
                        history_y = y_data[-history_length:]
                        assert len(history_x) == history_length
                        
                        if i == 0:
                            vel_x = [history_x[i] - history_x[i-1] for i in range(1, len(history_x))]
                            vel_y = [history_y[i] - history_y[i-1] for i in range(1, len(history_y))]
                        else:
                            vel_x = [(history_x[i] - history_x[i-1]) + np.random.normal(0, 1) for i in range(1, len(history_x))]
                            vel_y = [(history_y[i] - history_y[i-1]) + np.random.normal(0, 1) for i in range(1, len(history_y))]
                        
                        assert len(vel_x) == history_length-1
                        avg_vel_x = np.mean(vel_x)
                        avg_vel_y = np.mean(vel_y)
                        
                        pred_x = [x_data[-1] + i*avg_vel_x for i in range(1, 13)]
                        pred_y = [y_data[-1] + i*avg_vel_y for i in range(1, 13)]
                        assert len(pred_x) == 12
                        
                        all_pred_x.append(pred_x)
                        all_pred_y.append(pred_y)
                    

                    best_fde = None
                    best_ade = None
                    for i in range(len(all_pred_x)):
                        current_pred_x = all_pred_x[i]
                        current_pred_y = all_pred_y[i]

                        fde = calculate_FDE(current_pred_x, current_pred_y, x_gt, y_gt)
                        if best_fde == None or fde < best_fde:
                            best_fde = fde

                        ade = calculate_ADE(current_pred_x, current_pred_y, x_gt, y_gt)
                        if best_ade == None or ade < best_ade:
                            best_ade = ade

                    our_fde_best_of.append(best_fde)
                    our_ade_best_of.append(best_ade)
                    
    return our_fde_best_of, our_ade_best_of

def evaluate_cvm(data, dataset_title='', history_length=8):
    tot = 0
    all_fde = []
    all_ade = []

    for frame_id in tqdm(pd.unique(data.frame_id), desc='CVM - ' + dataset_title):

        frame_data = data[data.frame_id == frame_id]
        #print(frame_data)
        for node_id in pd.unique(frame_data.node_id):
            # Check if at least 7 historical points are present
            # PS: It might be so that the prediction starts at the 8th step instead of 7th? Edited the code to do this at the moment
            if len(data[((data.node_id == node_id) & (data.frame_id <= frame_id))]) >= 8:
                # Not sure why there has to be more than 12 frames to the future (at least 13) but it's the
                # only way to get the number of trajectron++ eval predictions to match up
                if len(data[((data.node_id == node_id) & (data.frame_id > frame_id))]) >= 12:
                    tot += 1
                    node_history_data = data[((data.node_id == node_id) & (data.frame_id <= frame_id) & (data.frame_id >= frame_id-7))]
                    node_gt_data = data[((data.node_id == node_id) & (data.frame_id > frame_id) & (data.frame_id <= frame_id+12))]

                    x_data = list(node_history_data.pos_x)
                    y_data = list(node_history_data.pos_y)
                    assert len(x_data) == 8

                    x_gt = list(node_gt_data.pos_x)
                    y_gt = list(node_gt_data.pos_y)
                    assert len(x_gt) == 12

                    history_x = x_data[-history_length:]
                    history_y = y_data[-history_length:]
                    assert len(history_x) == history_length

                    vel_x = [history_x[i] - history_x[i-1] for i in range(1, len(history_x))]
                    vel_y = [history_y[i] - history_y[i-1] for i in range(1, len(history_y))]

                    assert len(vel_x) == history_length-1
                    avg_vel_x = np.mean(vel_x)
                    avg_vel_y = np.mean(vel_y)

                    pred_x = [x_data[-1] + i*avg_vel_x for i in range(1, 13)]
                    pred_y = [y_data[-1] + i*avg_vel_y for i in range(1, 13)]
                    assert len(pred_x) == 12

                    fde = calculate_FDE(pred_x, pred_y, x_gt, y_gt)
                    ade = calculate_ADE(pred_x, pred_y, x_gt, y_gt)

                    all_fde.append(fde)
                    all_ade.append(ade)
                    
    return all_fde, all_ade


## Automated results comparison

In [165]:
def read_trajectron_data(trajectron_resultset_name, base_folder='./trajectron++/results_paper_version/', suffix='best_of'):
    trajectron_fde = []
    with open(base_folder + trajectron_resultset_name + '_fde_' + suffix + '.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            trajectron_fde.append(float(row['value']))

    trajectron_ade = []
    with open(base_folder + trajectron_resultset_name + '_ade_'+ suffix + '.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            trajectron_ade.append(float(row['value']))
            
    return trajectron_fde, trajectron_ade

In [166]:
def evaluate_all_datasets(our_method_params, datasets, trajectron_resultset_names, trajectron_ar3_resultset_names, evaluate_most_likely=False, evaluate_with_interactions=False):
    base_path = './raw_data/'

    ours_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}
    trajectron_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}
    trajectron_ar3_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}
    cvm_long_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}
    cvm_short_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}

    for dataset_idx, dataset in enumerate(datasets):
        our_fde_bo20, our_ade_bo20 = [], []
        our_fde_most_likely, our_ade_most_likely = [], []
        
        cvm_fde_bo20, cvm_ade_bo20 = [], []
        cvm_short_fde_bo20, cvm_short_ade_bo20 = [], []
        
        cvm_fde_ml, cvm_ade_ml = [], []
        cvm_short_fde_ml, cvm_short_ade_ml = [], []
        
        for scene_idx, scene in enumerate(dataset):
            data = pd.read_csv(base_path + scene, sep='\t', index_col=False, header=None)
            data.columns = ['frame_id', 'track_id', 'pos_x', 'pos_y']

            data = process_data(data)

            ## Ours
            our_fde_best_of_20, our_ade_best_of_20, our_fde_single, our_ade_single = evaluate_our_method(data, our_method_params[dataset_idx], dataset_title=trajectron_resultset_names[dataset_idx], smoothing=True)
            our_fde_bo20 += our_fde_best_of_20
            our_ade_bo20 += our_ade_best_of_20
            if evaluate_most_likely:
                our_fde_most_likely += our_fde_single
                our_ade_most_likely += our_ade_single

            ## CVM
            cvm_fde_best_of, cvm_ade_best_of = evaluate_cvm_with_scenarios(data, dataset_title=trajectron_resultset_names[dataset_idx])
            cvm_fde_bo20 += cvm_fde_best_of
            cvm_ade_bo20 += cvm_ade_best_of
            
            cvm_fde_short_history_best_of, cvm_ade_short_history_best_of = evaluate_cvm_with_scenarios(data, dataset_title=trajectron_resultset_names[dataset_idx], history_length=2)
            cvm_short_fde_bo20 += cvm_fde_short_history_best_of
            cvm_short_ade_bo20 += cvm_ade_short_history_best_of

            if evaluate_most_likely:
                cvm_fde, cvm_ade = evaluate_cvm(data, dataset_title=trajectron_resultset_names[dataset_idx])
                cvm_fde_ml += cvm_fde
                cvm_ade_ml += cvm_ade
                
                cvm_fde_short_history, cvm_ade_short_history = evaluate_cvm(data, dataset_title=trajectron_resultset_names[dataset_idx], history_length=2)
                cvm_short_fde_ml += cvm_fde_short_history
                cvm_short_ade_ml += cvm_ade_short_history

        # add our and CVM results to the data dict
        ours_results['BEST_OF_20']['FDE'].append(np.mean(our_fde_bo20))
        ours_results['BEST_OF_20']['ADE'].append(np.mean(our_ade_bo20))
        if evaluate_most_likely:
            ours_results['MOST_LIKELY']['FDE'].append(np.mean(our_fde_most_likely))
            ours_results['MOST_LIKELY']['ADE'].append(np.mean(our_ade_most_likely))
            
        cvm_long_results['BEST_OF_20']['FDE'].append(np.mean(cvm_fde_best_of))
        cvm_long_results['BEST_OF_20']['ADE'].append(np.mean(cvm_ade_best_of))
        cvm_short_results['BEST_OF_20']['FDE'].append(np.mean(cvm_fde_short_history_best_of))
        cvm_short_results['BEST_OF_20']['ADE'].append(np.mean(cvm_ade_short_history_best_of))
        if evaluate_most_likely:
            cvm_long_results['MOST_LIKELY']['FDE'].append(np.mean(cvm_fde))
            cvm_long_results['MOST_LIKELY']['ADE'].append(np.mean(cvm_ade))
            cvm_short_results['MOST_LIKELY']['FDE'].append(np.mean(cvm_fde_short_history))
            cvm_short_results['MOST_LIKELY']['ADE'].append(np.mean(cvm_ade_short_history))
                
        ## Trajectron
        trajectron_fde, trajectron_ade = read_trajectron_data(trajectron_resultset_names[dataset_idx])
        trajectron_results['BEST_OF_20']['FDE'].append(np.mean(trajectron_fde))
        trajectron_results['BEST_OF_20']['ADE'].append(np.mean(trajectron_ade))
        
        if evaluate_most_likely:
            trajectron_fde, trajectron_ade = read_trajectron_data(trajectron_resultset_names[dataset_idx], suffix='most_likely')
            trajectron_results['MOST_LIKELY']['FDE'].append(np.mean(trajectron_fde))
            trajectron_results['MOST_LIKELY']['ADE'].append(np.mean(trajectron_ade))
        
        ## Trajectron with interactions
        if evaluate_with_interactions:
            trajectron_ar3_fde, trajectron_ar3_ade = read_trajectron_data(trajectron_ar3_resultset_names[dataset_idx])
            trajectron_ar3_results['BEST_OF_20']['FDE'].append(np.mean(trajectron_ar3_fde))
            trajectron_ar3_results['BEST_OF_20']['ADE'].append(np.mean(trajectron_ar3_ade))

            if evaluate_most_likely:
                trajectron_ar3_fde, trajectron_ar3_ade = read_trajectron_data(trajectron_resultset_names[dataset_idx], suffix='most_likely')
                trajectron_ar3_results['MOST_LIKELY']['FDE'].append(np.mean(trajectron_ar3_fde))
                trajectron_ar3_results['MOST_LIKELY']['ADE'].append(np.mean(trajectron_ar3_ade))

        # make sure that there is no discrepancy between our data processing and trajectron evaluation results size
        if len(dataset) == 1:
            num_predictable_trajectories = get_total_predictable_slices(data)
            assert len(trajectron_fde) == num_predictable_trajectories
            assert len(trajectron_ade) == num_predictable_trajectories
    
    return [
        ours_results,
        trajectron_results,
        trajectron_ar3_results,
        cvm_long_results,
        cvm_short_results
    ]

### Running the evaluation

In [167]:
eth_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 300, 
    'CONST_VEL_MODEL_PROB': 0.5, 
    'STOP_PROB': 0.05, 
    'DISCOUNT_AVG_PROB': 1.0, 
    'DISCOUNT_LOWER_BOUND': 0.15, 
    'VELOCITY_CHANGE_PROB': 0.1,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.2, 
    'ANGLE_CHANGE_NOISE': 2, 
    'GROUP_PERCENTAGES': [0.05, 0.5, 0.75, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 9, 6, 4]
}

hotel_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 300, 
    'CONST_VEL_MODEL_PROB': 0.5, 
    'STOP_PROB': 0.05, 
    'DISCOUNT_AVG_PROB': 1.0, 
    'DISCOUNT_LOWER_BOUND': 0.1, 
    'VELOCITY_CHANGE_PROB': 0.1,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.2, 
    'ANGLE_CHANGE_NOISE': 2, 
    'GROUP_PERCENTAGES': [0.05, 0.60, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 11, 8]
}

zara1_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 300, 
    'CONST_VEL_MODEL_PROB': 0.5, 
    'STOP_PROB': 0.05, 
    'DISCOUNT_AVG_PROB': 1.0, 
    'DISCOUNT_LOWER_BOUND': 0.2, 
    'VELOCITY_CHANGE_PROB': 0.1,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.2, 
    'ANGLE_CHANGE_NOISE': 2, 
    'GROUP_PERCENTAGES': [0.05, 0.60, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 14, 5]
}

zara2_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 300, 
    'CONST_VEL_MODEL_PROB': 0.5, 
    'STOP_PROB': 0.05, 
    'DISCOUNT_AVG_PROB': 1.0, 
    'DISCOUNT_LOWER_BOUND': 0.2, 
    'VELOCITY_CHANGE_PROB': 0.1,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.1, 
    'ANGLE_CHANGE_NOISE': 2, 
    'GROUP_PERCENTAGES': [0.05, 0.60, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 14, 5]
}

univ_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 300, 
    'CONST_VEL_MODEL_PROB': 0.5, 
    'STOP_PROB': 0.05, 
    'DISCOUNT_AVG_PROB': 1.0, 
    'DISCOUNT_LOWER_BOUND': 0.1, 
    'VELOCITY_CHANGE_PROB': 0.1,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.2, 
    'ANGLE_CHANGE_NOISE': 2, 
    'GROUP_PERCENTAGES': [0.05, 0.5, 0.75, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 9, 6, 4]
}


our_method_params = [eth_params, hotel_params, univ_params, zara1_params, zara2_params]

datasets = [
    ['eth/test/biwi_eth.txt'], 
    ['hotel/test/biwi_hotel.txt'], 
    ['univ/test/students001.txt', 'univ/test/students003.txt'],
    ['zara1/test/crowds_zara01.txt'], 
    ['zara2/test/crowds_zara02.txt'],
]

trajectron_resultset_names = [
    'eth_vel', 
    'hotel_vel',
    'univ_vel',
    'zara1_vel_modified', 
    'zara2_vel_modified',
]

trajectron_ar3_resultset_names = [
    'eth_ar3', 
    'hotel_ar3', 
    'univ_ar3',
    'zara1_ar3', 
    'zara2_ar3',
]

res = evaluate_all_datasets(our_method_params, datasets, trajectron_resultset_names, trajectron_ar3_resultset_names, evaluate_most_likely=True)


CVM - zara2_vel_modified: 100%|█████████████████████████████████████████████████████████████| 1052/1052 [00:15<00:00, 68.98it/s]


In [169]:
ours_results = res[0]
trajectron_results = res[1]
trajectron_ar3_results = res[2]
cvm_long_results = res[3]
cvm_short_results = res[4]

In [170]:
sgan_results = {
    'BEST_OF_20': {
        'FDE': [1.52, 1.61, 0.69, 1.26, 0.84], 
        'ADE': [0.81, 0.72, 0.60, 0.34, 0.42]
    }, 
    'MOST_LIKELY': {
        'FDE': [2.21, 2.18, 1.28, 0.91, 1.11], 
        'ADE': [ 1.13, 1.01, 0.60, 0.42, 0.52]
    }
}

In [171]:
index = [
    'ETH', 
    'Hotel', 
    'Univ',
    'Zara 1', 
    'Zara 2',
    
]

df_data_best_of_20_fde = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['BEST_OF_20']['FDE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['BEST_OF_20']['FDE'], index = index),
    'Social-GAN': pd.Series(sgan_results['BEST_OF_20']['FDE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['BEST_OF_20']['FDE'], index = index),
    #'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['BEST_OF_20']['FDE'], index = index),
    'Ours': pd.Series(ours_results['BEST_OF_20']['FDE'], index = index)
}

df_best_of_20_fde = pd.DataFrame(df_data_best_of_20_fde)

df_data_best_of_20_ade = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['BEST_OF_20']['ADE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['BEST_OF_20']['ADE'], index = index),
    'Social-GAN': pd.Series(sgan_results['BEST_OF_20']['ADE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['BEST_OF_20']['ADE'], index = index),
    #'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['BEST_OF_20']['ADE'], index = index),
    'Ours': pd.Series(ours_results['BEST_OF_20']['ADE'], index = index)
}

df_best_of_20_ade = pd.DataFrame(df_data_best_of_20_ade)

df_data_most_likely_fde = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['MOST_LIKELY']['FDE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['MOST_LIKELY']['FDE'], index = index),
    'Social-GAN': pd.Series(sgan_results['MOST_LIKELY']['FDE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['MOST_LIKELY']['FDE'], index = index),
    #'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['MOST_LIKELY']['FDE'], index = index),
    'Ours': pd.Series(ours_results['MOST_LIKELY']['FDE'], index = index)
}

df_most_likely_fde = pd.DataFrame(df_data_most_likely_fde)

df_data_most_likely_ade = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['MOST_LIKELY']['ADE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['MOST_LIKELY']['ADE'], index = index),
    'Social-GAN': pd.Series(sgan_results['MOST_LIKELY']['ADE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['MOST_LIKELY']['ADE'], index = index),
    #'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['MOST_LIKELY']['ADE'], index = index),
    'Ours': pd.Series(ours_results['MOST_LIKELY']['ADE'], index = index)
}

df_most_likely_ade = pd.DataFrame(df_data_most_likely_ade)

### Best of 20 - FDE

In [172]:
# K-means without smoothing
df_best_of_20_fde.style.highlight_min(color = 'lightgreen', axis = 1)

,CVM (8pt history),CVM (2pt history),Social-GAN,Trajectron++,Ours
ETH,1.091436,1.698870,1.520000,0.812384,0.659920
Hotel,0.358949,0.581098,1.610000,0.197178,0.185460
Univ,0.966364,1.206381,0.690000,0.450190,0.445767
Zara 1,0.788392,0.885519,1.260000,0.341834,0.411453
Zara 2,0.553935,0.651882,0.840000,0.253352,0.304201


In [173]:
#print(df_best_of_20_fde.to_latex(header=False))

print(df_best_of_20_fde.style.highlight_min(props='textbf:--rwrap', axis=1).to_latex(hrules=True))

\begin{tabular}{lrrrrr}
\toprule
{} & {CVM (8pt history)} & {CVM (2pt history)} & {Social-GAN} & {Trajectron++} & {Ours} \\
\midrule
ETH & 1.091436 & 1.698870 & 1.520000 & 0.812384 & \textbf{0.659920} \\
Hotel & 0.358949 & 0.581098 & 1.610000 & 0.197178 & \textbf{0.185460} \\
Univ & 0.966364 & 1.206381 & 0.690000 & 0.450190 & \textbf{0.445767} \\
Zara 1 & 0.788392 & 0.885519 & 1.260000 & \textbf{0.341834} & 0.411453 \\
Zara 2 & 0.553935 & 0.651882 & 0.840000 & \textbf{0.253352} & 0.304201 \\
\bottomrule
\end{tabular}



### Best of 20 - ADE

In [174]:
# K-means without smoothing
df_best_of_20_ade.style.highlight_min(color = 'lightgreen', axis = 1)

,CVM (8pt history),CVM (2pt history),Social-GAN,Trajectron++,Ours
ETH,0.635120,0.868997,0.810000,0.396031,0.438772
Hotel,0.203495,0.309259,0.720000,0.115423,0.125524
Univ,0.525410,0.570101,0.600000,0.204697,0.263688
Zara 1,0.411510,0.407797,0.340000,0.154697,0.241797
Zara 2,0.296717,0.303092,0.420000,0.115362,0.183251


In [175]:
print(df_best_of_20_ade.style.highlight_min(props='textbf:--rwrap', axis=1).to_latex())

\begin{tabular}{lrrrrr}
{} & {CVM (8pt history)} & {CVM (2pt history)} & {Social-GAN} & {Trajectron++} & {Ours} \\
ETH & 0.635120 & 0.868997 & 0.810000 & \textbf{0.396031} & 0.438772 \\
Hotel & 0.203495 & 0.309259 & 0.720000 & \textbf{0.115423} & 0.125524 \\
Univ & 0.525410 & 0.570101 & 0.600000 & \textbf{0.204697} & 0.263688 \\
Zara 1 & 0.411510 & 0.407797 & 0.340000 & \textbf{0.154697} & 0.241797 \\
Zara 2 & 0.296717 & 0.303092 & 0.420000 & \textbf{0.115362} & 0.183251 \\
\end{tabular}



### Single output - FDE

In [176]:
df_most_likely_fde.style.highlight_min(color = 'lightgreen', axis = 1)

,CVM (8pt history),CVM (2pt history),Social-GAN,Trajectron++,Ours
ETH,2.303287,2.281890,2.210000,1.615308,1.911543
Hotel,0.462310,0.614198,2.180000,0.498741,0.752817
Univ,1.576302,1.368753,1.280000,1.204670,1.202177
Zara 1,1.131947,0.952377,0.910000,0.769595,1.466909
Zara 2,0.859903,0.724414,1.110000,0.589387,0.788144


In [177]:
print(df_most_likely_fde.style.highlight_min(props='textbf:--rwrap', axis=1).to_latex())

\begin{tabular}{lrrrrr}
{} & {CVM (8pt history)} & {CVM (2pt history)} & {Social-GAN} & {Trajectron++} & {Ours} \\
ETH & 2.303287 & 2.281890 & 2.210000 & \textbf{1.615308} & 1.911543 \\
Hotel & \textbf{0.462310} & 0.614198 & 2.180000 & 0.498741 & 0.752817 \\
Univ & 1.576302 & 1.368753 & 1.280000 & 1.204670 & \textbf{1.202177} \\
Zara 1 & 1.131947 & 0.952377 & 0.910000 & \textbf{0.769595} & 1.466909 \\
Zara 2 & 0.859903 & 0.724414 & 1.110000 & \textbf{0.589387} & 0.788144 \\
\end{tabular}



### Single output - ADE

In [178]:
df_most_likely_ade.style.highlight_min(color = 'lightgreen', axis = 1)

,CVM (8pt history),CVM (2pt history),Social-GAN,Trajectron++,Ours
ETH,1.101935,1.075458,1.130000,0.694815,0.947190
Hotel,0.243328,0.319356,1.010000,0.224216,0.354401
Univ,0.781078,0.618222,0.600000,0.467850,0.562417
Zara 1,0.551512,0.427223,0.420000,0.297096,0.622456
Zara 2,0.421038,0.323937,0.520000,0.227388,0.371437


In [179]:
print(df_most_likely_ade.style.highlight_min(props='textbf:--rwrap', axis=1).to_latex())

\begin{tabular}{lrrrrr}
{} & {CVM (8pt history)} & {CVM (2pt history)} & {Social-GAN} & {Trajectron++} & {Ours} \\
ETH & 1.101935 & 1.075458 & 1.130000 & \textbf{0.694815} & 0.947190 \\
Hotel & 0.243328 & 0.319356 & 1.010000 & \textbf{0.224216} & 0.354401 \\
Univ & 0.781078 & 0.618222 & 0.600000 & \textbf{0.467850} & 0.562417 \\
Zara 1 & 0.551512 & 0.427223 & 0.420000 & \textbf{0.297096} & 0.622456 \\
Zara 2 & 0.421038 & 0.323937 & 0.520000 & \textbf{0.227388} & 0.371437 \\
\end{tabular}

